In [ ]:
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
import wandb
import pickle

In [ ]:
wandb.init(project="categorize-sscorpus")

In [ ]:
# Define the categories in which SSCORPUS will be categorized

labels = ["Business", "Entertainment", "Geography", "Politics", "Science", "Sports"]

In [ ]:
# Initialize the classifier

classifier = pipeline("zero-shot-classification", device=0)

In [ ]:
# Load the SSCORPUS containing Levenshtein scores

columns = ['Simple', 'Professional', 'Levenshtein']
path = "1_sscorpus_levenshtein_calculated/sscorpus_levenshtein_final.csv"
df_levenshtein = pd.read_csv(path, sep=',', header=None, names=columns)

simple_list = df_levenshtein['Simple'].values.tolist()
professional_list = df_levenshtein['Professional'].values.tolist()

In [ ]:
# Categorize the SSCORPUS

# set the multi_label
multi_label = True
batch_size = 10
results = []
for i in tqdm(range(0, len(professional_list), batch_size)):
    results += classifier(professional_list[i:i+batch_size], labels, multi_label=multi_label)

In [ ]:
# Dump the categorized SSCORPUS on a file

label = ""
if multi_label:
    label = "multi_label_true"
else:
    label = "multi_label_false"

dump_path = "2_sscorpus_categorized/"+label+"/scorpus_categorized_raw"
fout = open(dump_path, 'wb')
pickle.dump(results, fout)
fout.close()